<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ux_AYLctM3qleYbXyUKE-ZF0m9KOBuF1?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- Join Innovation Community

Learn by building. Get expert mentorship and work on real AI projects.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




## Structured Outputs from OpenAI Client

Notebook by [Build Fast with AI](https://www.buildfastwithai.com/)

In [ ]:
!pip install -qU openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from pydantic import BaseModel
from openai import OpenAI

class UserInfo(BaseModel):
  name: str
  age: int

client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Satvik is 18 years old"}],
    response_format=UserInfo,
)

message = completion.choices[0].message
print(message.parsed.name)
print(message.parsed.age)

Satvik
18


In [ ]:
from pydantic import BaseModel

from openai import OpenAI


class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: list[Step]
    final_answer: str


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=MathResponse,
)

message = completion.choices[0].message

In [ ]:
message.content

'{"steps":[{"explanation":"First, subtract 31 from both sides of the equation to isolate the term with the variable on one side.","output":"8x + 31 - 31 = 2 - 31"},{"explanation":"Simplify both sides. The left side becomes 8x and the right side becomes -29.","output":"8x = -29"},{"explanation":"Next, divide both sides by 8 to solve for x.","output":"x = \\\\frac{-29}{8}"}],"final_answer":"x = -\\\\frac{29}{8}"}'

In [ ]:
message.parsed

MathResponse(steps=[Step(explanation='Subtract 31 from both sides to isolate the term with x.', output='8x + 31 - 31 = 2 - 31')], final_answer='8x = -29')

In [ ]:
message.parsed

In [ ]:
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)

[Step(explanation='First, subtract 31 from both sides of the equation to isolate the term with the variable on one side.', output='8x + 31 - 31 = 2 - 31'), Step(explanation='Simplify both sides. The left side becomes 8x and the right side becomes -29.', output='8x = -29'), Step(explanation='Next, divide both sides by 8 to solve for x.', output='x = \\frac{-29}{8}')]
x = -\frac{29}{8}


In [ ]:
from enum import Enum
from typing import Union

from pydantic import BaseModel

import openai
from openai import OpenAI


class Table(str, Enum):
    orders = "orders"
    customers = "customers"
    products = "products"


class Column(str, Enum):
    id = "id"
    status = "status"
    expected_delivery_date = "expected_delivery_date"
    delivered_at = "delivered_at"
    shipped_at = "shipped_at"
    ordered_at = "ordered_at"
    canceled_at = "canceled_at"


class Operator(str, Enum):
    eq = "="
    gt = ">"
    lt = "<"
    le = "<="
    ge = ">="
    ne = "!="


class OrderBy(str, Enum):
    asc = "asc"
    desc = "desc"


class DynamicValue(BaseModel):
    column_name: str


class Condition(BaseModel):
    column: str
    operator: Operator
    value: Union[str, int, DynamicValue]


class Query(BaseModel):
    table_name: Table
    columns: list[Column]
    conditions: list[Condition]
    order_by: OrderBy


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. The current date is August 6, 2024. You help users query for the data they are looking for by calling the query function.",
        },
        {
            "role": "user",
            "content": "look up all my orders in may of last year that were fulfilled but not delivered on time",
        },
    ],
    tools=[
        openai.pydantic_function_tool(Query),
    ],
)

print(completion.choices[0].message.tool_calls[0].function.parsed_arguments)

table_name=<Table.orders: 'orders'> columns=[<Column.id: 'id'>, <Column.status: 'status'>, <Column.expected_delivery_date: 'expected_delivery_date'>, <Column.delivered_at: 'delivered_at'>, <Column.shipped_at: 'shipped_at'>, <Column.ordered_at: 'ordered_at'>, <Column.canceled_at: 'canceled_at'>] conditions=[Condition(column='shipped_at', operator=<Operator.ge: '>='>, value='2023-05-01'), Condition(column='shipped_at', operator=<Operator.le: '<='>, value='2023-05-31'), Condition(column='status', operator=<Operator.eq: '='>, value='fulfilled'), Condition(column='delivered_at', operator=<Operator.gt: '>'>, value=DynamicValue(column_name='expected_delivery_date'))] order_by=<OrderBy.asc: 'asc'>
